# PDF Survey Digitizer (Google Sheets Integration)

## First make sure the requests and google sheets libraries are installed

In [ ]:
!pip install requests
!pip install --upgrade gspread

## Questions for the LLM to extract from the image

In [12]:
questions = [
    "Date:",
    "Homeowner's Name:",
    "Please share an example of how these repairs have made an impact on your day-to-day life:",
    "Please explain why you rated the previous question this way:",
    "For previous question please explain why or why not? "
]


## This code uploads the pdf to the Azure Document AI and recieves the content of the pdf in json

In [7]:
# prompt: get a pdf from my google drive

if not os.path.exists('/content/drive'):
  from google.colab import drive
  drive.mount('/content/drive')
  import os

pdf_dir = '/content/drive/MyDrive/Demo/DocScanner/ScannedSurveys/'

for file in os.listdir(pdf_dir):
  if file.endswith(".pdf"):
    filename = os.path.join(pdf_dir, file)
    print(filename)


/content/drive/MyDrive/Demo/DocScanner/ScannedSurveys/HomePreservationProgram.pdf


In [8]:
import time
import json
import requests


from google.colab import userdata
config = userdata.get('config')
config = json.loads(config)

# Define your endpoint and API key
endpoint = config["document-ai"]["endpoint"]
api_key = config["document-ai"]["api-key"]

# Define the URL for document analysis
url = f"{endpoint}/formrecognizer/v2.1/layout/analyze"

# Set the headers
headers = {
    "Ocp-Apim-Subscription-Key": api_key,
    "Content-Type": "application/pdf",  # Or "application/json" if using JSON
}

# Open and send the document (PDF or image)
with open(filename, "rb") as f:
    data = f.read()

# Make the request
response = requests.post(url, headers=headers, data=data)

# Check if the request was accepted
if response.status_code == 202:
    print("Request accepted, processing...")
    # Get the operation URL from the response headers or JSON response
    operation_url = response.headers["Operation-Location"]  # Example of how it's returned

    # Poll the operation URL to check the status
    status = ""
    while status != "succeeded":
        response = requests.get(operation_url, headers={"Ocp-Apim-Subscription-Key": api_key})
        result = response.json()
        status = result["status"]

        # Wait for a few seconds before polling again
        time.sleep(5)
        print(f"Current status: {status}")

    # Once the status is "succeeded", you can get the results
    print("Processing complete!")
    with open("output.json", "w") as f:
        json.dump(result, f, indent=2)

else:
    print(f"Error: {response.status_code}, {response.text}")


{'document-ai': {'endpoint': 'https://hfhscanner.cognitiveservices.azure.com/', 'api-key': 'e2cb02246707415aae2e12bae69667d0'}, 'open-ai-llm': {'endpoint': 'https://hfhform.openai.azure.com/openai/deployments/gpt-35-turbo/chat/completions?api-version=2024-08-01-preview', 'api-key': '08fecceff2c74bd8b338a2c7cd97729d'}}
Request accepted, processing...
Current status: running
Current status: running
Current status: succeeded
Processing complete!


## Collects the lines from the json into paragraph form fo the LLM doesn't need as many tokens

In [9]:
import json

# Load the JSON data from the Azure response
with open("output.json", "r") as f:
    data = json.load(f)

# Initialize a list to hold lines
lines = []

# Assuming the structure has a 'analyzeResult' key with 'readResults'
for page in data.get("analyzeResult", {}).get("readResults", []):
    for line in page.get("lines", []):
        # Extract the text and add it to the list
        lines.append(line["text"])

# Combine the words into a single string
combined_text = " ".join(lines)

# Print the combined text (for debugging)
print(combined_text)

with open("output.txt", "w") as f:
    f.write(combined_text)

San Gabriel Valley Habitat for Humanity' Home Preservation Program Exit Survey This survey helps ensure we are on track with our goals and allows us to make improvements to the program for future applicants. Thank you for taking the time to fill this out and feel free to get in touch if you have questions or more to say. Oct 12th 2024 John Pork Today's Date Homeowner's Name 1. Over the past twelve (12) months, how often did you feel stressed and/or nervous about home repairs/maintenance? Always Often Sometimes Rarely Never 2. What has prevented you and your household from making the repairs previously? (Select all that apply) Health-related issues Lack of expertise/access to expertise Lack of funds Other: 3. Did the completed repairs help in any of the following ways? (If yes, circle all that apply) Improved mobility Increased accessibility Increased safety Other: Please share an example of how these repairs have made an impact on your day-to-day life: The repairs fixed my driveway, no

## This code sends the pdf file data and the survey questions to chatGPT

In [30]:
import requests
import json
import time
from requests.exceptions import RequestException

# Load credentials
from google.colab import userdata
config = userdata.get('config')
config = json.loads(config)

# Define your endpoint and API key
endpoint = config["open-ai-llm"]["endpoint"]
api_key = config["open-ai-llm"]["api-key"]

# Define the headers, including the API key for authentication
headers = {
    "Content-Type": "application/json",
    "api-key": api_key
}

with open('output.txt', 'r') as file:
    pdf_data = file.read()
    print("PDF data:" + str(pdf_data))

# Define the payload (prompt, model, etc.)

# Function to send the request and check for the response in a loop
def prompt_GPT(question):
    data = {
        "model": "gpt-3.5-turbo",
        "messages": [
            {
                "role": "system", "content": "You are a data extractor."
            },
            {
                "role": "user", "content": f"""
                You are provided with a long text containing questions and answers. Your task is to extract the answer to a given question making inferences because the questions might be worded differently. Format any dates as mm/dd/yyyy. Only return the answer text without extra explanation.

                Here is the long text:
                "{pdf_data}"

                Extract the answer to the following question:
                "{question}"

                Return only the answer in plain text as a response because I have code to pair your response with the question in a python dictionary in this format:
                {{
                    "question": "your exact response"
                }}

                Dont forget that if you notice a date, you should respond with the formatted date in the mm/dd/yyyy format.
                """
            }
        ],
        "temperature": 0.9,  # Control randomness: 0 = deterministic, 1 = more creative
        "max_tokens": 100    # Limit the response length
      }

    response_received = False
    attempt_count = 0

    while not response_received:
        try:
            print(f"Attempt {attempt_count + 1}: Sending request...")
            response = requests.post(
                f"{endpoint}",
                headers=headers, json=data
            )

            if response.status_code == 200:
                response_data = response.json()["choices"][0]["message"]["content"]
                print(f"Response: {response_data}")
                return response_data
                response_received = True  # Break the loop when response is successfully received
            else:
                print(f"Error: {response.status_code}")
                print(f"Message: {response.text}")

            # Increment attempt counter
            attempt_count += 1
            # Wait before trying again (to avoid overwhelming the server)
            time.sleep(2)

        except RequestException as e:
            print(f"An error occurred: {e}")
            time.sleep(2)  # Wait before retrying


response = {}

# Ensure questions is defined from an earlier code block
for question in questions:
    response[question] = prompt_GPT(question)

# Save the response to a JSON file
with open("response.json", "w") as f:
    json.dump(response, f, indent=2)

print("Complete.")


PDF data:San Gabriel Valley Habitat for Humanity' Home Preservation Program Exit Survey This survey helps ensure we are on track with our goals and allows us to make improvements to the program for future applicants. Thank you for taking the time to fill this out and feel free to get in touch if you have questions or more to say. Oct 12th 2024 John Pork Today's Date Homeowner's Name 1. Over the past twelve (12) months, how often did you feel stressed and/or nervous about home repairs/maintenance? Always Often Sometimes Rarely Never 2. What has prevented you and your household from making the repairs previously? (Select all that apply) Health-related issues Lack of expertise/access to expertise Lack of funds Other: 3. Did the completed repairs help in any of the following ways? (If yes, circle all that apply) Improved mobility Increased accessibility Increased safety Other: Please share an example of how these repairs have made an impact on your day-to-day life: The repairs fixed my dri

## Next we populate the row in the Google sheet with the survey data

In [33]:
# prompt: can you use the data in responses.json to populate a google sheet. the keys should be the names of the name of the column and append a row to inpiut this data

from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

import time

gc = gspread.authorize(creds)

# Open the spreadsheet by name or URL
sh = gc.open('Responses')

# Select the worksheet
worksheet = sh.get_worksheet(2) # Exit Survey
header_row = worksheet.row_values(1)
last_row = len(worksheet.get_all_values()) + 1


# Load the responses from the JSON file
with open("response.json", "r") as f:
    response_data = json.load(f)

response_data["Timestamp"] = time.strftime("%Y-%m-%d %H:%M:%S")
response_data["Email Address"] = "N/A"

for question, answer in response_data.items():
    try:
      column_index = header_row.index(question) + 1
      worksheet.update_cell(last_row, column_index, answer)
    except ValueError:
      print(f"Column '{question}' not found in the header row.")

print("Finished adding survey to the google sheet")

Finished adding survey to the google sheet
